In [ ]:
args = ["https://nexus.geomatys.com/repository/raw-public/testbed16/S2B_MSIL1C_20181105T062449_N0206_R091_T40KEC_20181105T093258.zip","https://nexus.geomatys.com/repository/raw-public/testbed16/S2B_MSIL1C_20181105T062449_N0206_R091_T40KED_20181105T093258.zip"]

In [ ]:
#
# library ADES  Client
#
import requests
import time

def ExecuteADES(processCode, inputCode, inputRef):
    
    if (len(inputRef) == 1):
        inputBlock = {
                        "id": inputCode,
                        "input": {
                            "format": {
                                "mimeType": "application/zip"
                            },
                            "value": {
                                "href": inputRef[0]
                            }
                        }
                    }
        inputBlocks = [inputBlock]
    else:
        i = 0
        inputBlocks = []
        while i < len(inputRef):
            inbk = {
                        "id": inputCode,
                        "input": {
                            "format": {
                                "mimeType": "application/zip"
                            },
                            "value": {
                                "href": inputRef[i]
                            }
                        }
                    }
            inputBlocks.append(inbk)
            i += 1
            
    
    request = {
        "inputs": inputBlocks,
        "outputs": [
            {
                "id": "output",
                "transmissionMode": "reference"
            }
        ],
        "mode": "async",
        "response": "document"
    }
    response = requests.post('https://tb16.geomatys.com/ades/WS/wps/default/processes/' + processCode + '/jobs', json=request)

    #print("Status code: ", response.status_code)
    print("ADES status location: " + response.headers["Location"])

    location = response.headers["Location"]

    response = requests.get(location)
    #print("Status code: ", response.status_code)
    status = response.json()["status"]
    print("ADES response status:" + status)
    print("waiting for ADES process completion.......")
    while (status == 'running') :
        time.sleep(1)
        response = requests.get(location)
        status = response.json()["status"]
        #print(status)

    if (status == 'succeeded'):
        response = requests.get(location + '/result')
        #print(response.json())
        outputs = response.json()["outputs"]
        results = []
        j = 0
        while j < len(outputs):
            result = outputs[j]["value"]["href"]
            print("result: " + result)
            results.append(result)
            j += 1
        return results


In [ ]:
#
# step 1: ADES process NDVI on sentinel input
#
ndvi = ExecuteADES('NDVIMultiSensor', 'files', args)


In [ ]:
#
# show intermediate results (optional)
#
from osgeo import gdal
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import uuid
import numpy as np

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams["xtick.color"] = "none"
plt.rcParams["ytick.color"] = "none"
plt.show()

for p in ndvi:
    if p.startswith(("http")):
        dlpath = "/tmp/" + str(uuid.uuid4())
        url = p
        ext = ''
        if '.' in url: ext = '.' + url.rsplit('.', 1)[1]
        r = requests.get(url, allow_redirects=True)
        open(dlpath + ext, 'wb').write(r.content)
        p = dlpath + ext
        #print(p)
        
    ndvitiff = gdal.Open(p)
    nparray = ndvitiff.GetRasterBand(1).ReadAsArray().astype(np.float32)
    
    plt.title('Calculated NDVI')
    plt.imshow(nparray, cmap=plt.cm.Greys)
    plt.show()


In [ ]:
#
# step 2: ADES process stack on NDVI input
#
stacked = ExecuteADES('NDVIStacker', 'files', ndvi)


In [ ]:
#
# prepare final result
#
import os

path = str(Path.home()) + "/outputs/"

if not os.path.exists(path):
    os.makedirs(path)

dlRes = []
for p in stacked:
    dlpath = path + str(uuid.uuid4())
    url = p
    ext = ''
    if '.' in url: ext = '.' + url.rsplit('.', 1)[1]
    r = requests.get(url, allow_redirects=True)
    open(dlpath + ext, 'wb').write(r.content)
    p = dlpath + ext
    print(p)
    dlRes.append(p)

In [ ]:
#
# show final result (optional)
#
from osgeo import gdal
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import uuid
import numpy as np

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams["xtick.color"] = "none"
plt.rcParams["ytick.color"] = "none"
plt.show()

for p in dlRes:
    ndvitiff = gdal.Open(p)
    nparray = ndvitiff.GetRasterBand(1).ReadAsArray().astype(np.float32)
    
    plt.title('Stacked NDVI')
    plt.imshow(nparray, cmap=plt.cm.Greys)
    plt.show()
